# RNN Basics

In [1]:
import tensorflow as tf
import numpy as np
import pprint as pp

### One cell RNN : input_dim (4) to output_dim (2)

![rnn link](http://i.imgur.com/Wl5Q0Ax.png)

input shape = (1,1,4)에서 마지막 숫자 4는 **input_dim**을 의미한다.

One hot encoding을 통해 h,e,l,o 4의 dim이 만들어졌다.

hidden_size는 **output_dim**을 의미한다.

In [12]:
hidden_size = 2
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

In [15]:
x_data = np.array([[[1,0,0,0]]], dtype=np.float32)

with tf.variable_scope('first'):
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=np.float32)
# 이후에도 같은 tf.nn.dynamic_rnn()을 반복해야하므로, 각각의 변수에 서로 다른 scope를 지정해줘야 한다.

In [16]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 0.19572318, -0.62309188]]], dtype=float32)


### One cell RNN : input_dim (4) to output_dim (2). sequence (5)

![rnn link](http://i.imgur.com/NlBUNTv.png)

shape = (1,5,4)에서 가운데 숫자 5는 **sequence_length**를 의미한다.

**sequence_length**가 1인 경우에는 하나의 character  'h'  가(one hot vector 형식) input이였지만,

**sequence_length**가 5인 경우에는 하나의 단어  'hello'  (one hot vector 형식)이 input이다.

In [6]:
hidden_size = 2
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

In [17]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

x_data = np.array([[h,e,l,l,o]], dtype=np.float32)

with tf.variable_scope('second'):
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=np.float32)

pp.pprint(x_data)
print(x_data.shape)

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
(1, 5, 4)


In [18]:
with tf.Session() as sess:
        
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape)

array([[[-0.46433732, -0.43623033],
        [ 0.40526074,  0.57321656],
        [ 0.40748829,  0.00102429],
        [ 0.29816294,  0.21091184],
        [ 0.09415741, -0.08221801]]], dtype=float32)
(1, 5, 2)


### One cell RNN : input_dim (4) to output_dim (2). sequence (5), batch (3)

![rnn link](http://i.imgur.com/rdiUvY3.png)

shape = (3,5,4)에서 첫번째 숫자 3은 **batch_size**를 의미한다.

**batch_size**가 1인 경우에는 하나의 단어 'hello' (one hot vector 형식)이 input이였지만,

**batch_size**가 3인 경우네는 여러개의 단어 ['hello', 'eolll', 'lleel'] (one hot vector 형식)이 input이다.

In [20]:
hidden_size = 2
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

In [21]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

x_data = np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)

with tf.variable_scope('third'):
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=np.float32)

pp.pprint(x_data)
print(x_data.shape)

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
(3, 5, 4)


In [23]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape)

array([[[ 0.03896408, -0.66833478],
        [ 0.62383968, -0.504884  ],
        [-0.22676456, -0.07495562],
        [-0.41680902, -0.53328949],
        [ 0.63910121, -0.46978766]],

       [[ 0.35153845, -0.38266632],
        [ 0.57481974,  0.10581679],
        [-0.51027781,  0.05430469],
        [-0.46773437, -0.64794487],
        [-0.12402228, -0.73103923]],

       [[-0.45467433, -0.3839013 ],
        [-0.2629064 , -0.69072896],
        [ 0.63588417, -0.65719122],
        [ 0.61060524, -0.10567395],
        [-0.42066422,  0.02359969]]], dtype=float32)
(3, 5, 2)


# 'Hi Hello' RNN 학습시키기

![rnn link](http://i.imgur.com/YjxQhW5.png)

In [24]:
hidden_size = 5 # output size == input size
input_dim = 5 # h, i, e, l, o
batch_size = 1 # 단어 1개 학습
sequence_length = 6 # 'hihell' 학습 -> 'ihello' 결과

In [25]:
idx2char = ['h','i','e','l','o'] # h = 0, i = 1, e = 2, l = 3, o = 4 후에 output 및 y_data를 문자로 변환시킬 때 사용한다.
x_data = [[0,1,0,2,3,3]] # 'hihell'
x_one_hot = [[[1,0,0,0,0],
              [0,1,0,0,0],
              [1,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,1,0],
              [0,0,0,1,0]]]

y_data = [[1,0,2,3,3,4]] # 'ihello'
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim]) # x_one_hot 형태
Y = tf.placeholder(tf.int32, [None, sequence_length]) # y_data 형태

In [27]:
cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32) # cell의 초기값을 0으로 
with tf.variable_scope('fourth'):
    outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype = np.float32)

In [29]:
weights = tf.ones([batch_size, sequence_length]) # [1, 6]

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights = weights) # 각 sequence 별 loss를 구해준다.
loss = tf.reduce_mean(sequence_loss) # 각 sequence 별 loss의 평균
train = tf.train.AdamOptimizer(learning_rate = 0.1).minimize(loss)

In [32]:
prediction = tf.argmax(outputs, axis = 2) # argmax는 최대값의 위치를 반환한다. axis=2이므로, input shape와 같은 형태로 output을 반환한다.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2001):
        l, _ = sess.run([loss, train], feed_dict = {X : x_one_hot, Y : y_data}) # loss를 계산하는데는 X,Y 둘다 필요
        result = sess.run(prediction, feed_dict = {X : x_one_hot}) # prediction을 계산하는데는 X만 필요
        
        result_str = [idx2char[c] for c in np.squeeze(result)] # result 숫자를 character로 바꿔준다. list comprehension
        if i%10 == 0:
            print(i, 'loss: ', l, 'prediction: ', result, 'true Y: ', y_data, 'prediction str: ', ''.join(result_str ))

0 loss:  1.54058 prediction:  [[2 0 4 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  eholll
10 loss:  0.880212 prediction:  [[2 0 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ehlllo
20 loss:  0.693573 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
30 loss:  0.600668 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
40 loss:  0.555302 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
50 loss:  0.49161 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
60 loss:  0.471676 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
70 loss:  0.466696 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
80 loss:  0.462474 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
90 loss:  0.46052 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] predicti

850 loss:  0.44176 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
860 loss:  0.441705 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
870 loss:  0.44165 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
880 loss:  0.441597 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
890 loss:  0.441545 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
900 loss:  0.441493 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
910 loss:  0.441442 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
920 loss:  0.441392 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
930 loss:  0.441343 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
940 loss:  0.441295 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 

1720 loss:  0.438878 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1730 loss:  0.438857 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1740 loss:  0.438837 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1750 loss:  0.438817 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1760 loss:  0.438797 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1770 loss:  0.438777 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1780 loss:  0.438757 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1790 loss:  0.438737 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1800 loss:  0.438718 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1810 loss:  0.438699 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 

대략 20번째 step부터 원하는 label이 나왔음을 확인할 수 있다.